In [5]:

# 连接mysql 使用pandas
#http://blog.csdn.net/u011715678/article/details/48497291  pandas和mysql双向交互
import pandas as pd
import MySQLdb
mysql_conn= MySQLdb.connect(host='tunnel.qydev.com', port=40100,user='root', passwd='', db='mytest')
#mysql_cn= MySQLdb.connect(host='127.0.0.1', port=3306,user='root', passwd='', db='mytest')
#df = pd.read_sql('select * from auth_user limit 100;', con=mysql_conn)  
#df = pd.read_sql('select * from Customers limit 10;', con=mysql_conn)    
#mysql_cn.close()


In [11]:
sql = "show tables"
pd.read_sql(sql, con=mysql_conn)

,Tables_in_mytest
0,Customers
1,OrderItems
2,Orders
3,Products
4,Vendors


In [12]:
pd.read_sql('select * from Customers limit 10;', con=mysql_conn)

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
2,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
3,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com
4,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None


In [21]:
#第四章：过滤数据
sql = '''SELECT prod_name, prod_price
FROM Products
-- WHERE prod_price = 3.49; 
WHERE prod_price < 10
ORDER BY prod_name;
'''
pd.read_sql(sql, con=mysql_conn)

,prod_name,prod_price
0,12 inch teddy bear,8.99
1,8 inch teddy bear,5.99
2,Bird bean bag toy,3.49
3,Fish bean bag toy,3.49
4,King doll,9.49
5,Queen doll,9.49
6,Rabbit bean bag toy,3.49
7,Raggedy Ann,4.99


In [23]:
# 第五章：通配符过滤
sql = '''SELECT prod_id,prod_name
FROM Products
WHERE prod_name LIKE 'Fish%'
'''
pd.read_sql(sql, con=mysql_conn)

,prod_id,prod_name
0,BNBG01,Fish bean bag toy


In [24]:
# 第七章：执行计算字段
sql = '''SELECT prod_id,quantity,item_price,
quantity*item_price AS expaned_price
FROM OrderItems
WHERE order_num = 20008;
'''
pd.read_sql(sql, con=mysql_conn)

,prod_id,quantity,item_price,expaned_price
0,RGAN01,5,4.99,24.95
1,BR03,5,11.99,59.95
2,BNBG01,10,3.49,34.90
3,BNBG02,10,3.49,34.90
4,BNBG03,10,3.49,34.90


In [26]:
#第八章：使用数据处理函数
sql = '''SELECT *
FROM Orders
WHERE YEAR(order_date) = 2004;
'''
pd.read_sql(sql, con=mysql_conn)

,order_num,order_date,cust_id
0,20005,2004-05-01,1000000001
1,20006,2004-01-12,1000000003
2,20007,2004-01-30,1000000004
3,20008,2004-02-03,1000000005
4,20009,2004-02-08,1000000001


In [31]:
#第九章：汇总数据
sql = '''SELECT AVG(prod_price) AS avg_price 
FROM Products;
'''
pd.read_sql(sql, con=mysql_conn)

,avg_price
0,6.823333


In [33]:
#第九章：汇总数据
sql = '''SELECT COUNT(*) AS num_cust 
FROM Customers;
'''
pd.read_sql(sql, con=mysql_conn)

#其他的函数 MAX MIN SUM

,num_cust
0,5


In [34]:
#第九章：汇总数据
sql = '''SELECT SUM(item_price*quantity) AS total_price 
FROM OrderItems
WHERE order_num = 20005;
'''
pd.read_sql(sql, con=mysql_conn)

#其他的函数 MAX MIN SUM

,total_price
0,1648


In [37]:
# 第十章：分组数据
sql = '''SELECT vend_id,COUNT(*) AS num_prods
FROM Products
GROUP BY vend_id;
'''
pd.read_sql(sql, con=mysql_conn)


,vend_id,num_prods
0,BRS01,3
1,DLL01,4
2,FNG01,2


In [40]:
### 第十章：分组数据:过滤分组
sql = '''SELECT cust_id,COUNT(*) AS orders
FROM Orders
GROUP BY cust_id
-- HAVING orders >= 2;
HAVING COUNT(*) >=2;
'''
pd.read_sql(sql, con=mysql_conn)


,cust_id,orders
0,1000000001,2


In [41]:
#第十一章：使用子查询
sql = '''SELECT cust_id 
FROM Orders
WHERE order_num in (SELECT order_num
FROM OrderItems
WHERE prod_id = 'RGAN01'
);
'''
pd.read_sql(sql, con=mysql_conn)


,cust_id
0,1000000004
1,1000000005


In [47]:
#第十二章：联结表
sql = '''SELECT vend_name,prod_name,prod_price 
-- FROM Vendors,Products
-- WHERE Vendors.vend_id = Products.vend_id;
FROM Vendors INNER JOIN Products
ON Vendors.vend_id = Products.vend_id;
-- 也可以联接3个表
'''
pd.read_sql(sql, con=mysql_conn)


,vend_name,prod_name,prod_price
0,Bears R Us,8 inch teddy bear,5.99
1,Bears R Us,12 inch teddy bear,8.99
2,Bears R Us,18 inch teddy bear,11.99
3,Doll House Inc.,Fish bean bag toy,3.49
4,Doll House Inc.,Bird bean bag toy,3.49
5,Doll House Inc.,Rabbit bean bag toy,3.49
6,Doll House Inc.,Raggedy Ann,4.99
7,Fun and Games,King doll,9.49
8,Fun and Games,Queen doll,9.49


In [49]:
# 第十四章： 组合查询
sql = '''SELECT cust_name,cust_contact,cust_email 
FROM Customers
WHERE cust_state IN ('IL','IN','MI')
UNION

SELECT cust_name,cust_contact,cust_email 
FROM Customers
WHERE cust_name = 'Fun4All'
-- 默认情况下去掉重复行
'''
pd.read_sql(sql, con=mysql_conn)


,cust_name,cust_contact,cust_email
0,Village Toys,John Smith,sales@villagetoys.com
1,Fun4All,Jim Jones,jjones@fun4all.com
2,The Toy Store,Kim Howard,None
3,Fun4All,Denise L. Stephens,dstephens@fun4all.com
